In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime
from glob import glob
from astropy.time import Time

import sys
sys.path.insert(0, '../')

In [9]:
from astroplan import EclipsingSystem
from libra import trappist1, kepler296, kepler62, transit_duration
import json

for cycle in ['cycle0', 'cycle1']:
    target_names = ['Kepler-296', 'Kepler-62', 'TRAPPIST-1']
    target_paths = sorted(glob('../libra/data/apt/{0}/*visibility.txt'.format(cycle)))

    for target, path in zip(target_names, target_paths):
        raw_text = open(path).read().splitlines()
        dates = [[d.strip() for d in row.split('\t')[0].split('-')] for row in raw_text]
        datetimes = [[datetime.strptime(d, '%d %b %Y') for d in row] for row in dates]

        i = 0
        milestone_dates = []
        while i <= len(datetimes) - 1:
            span = []
            if i == 0: 
                milestone_dates.append(datetimes[i][0])
            elif i == len(datetimes) - 1: 
                milestone_dates.append(datetimes[i][1])
            elif datetimes[i][1] != datetimes[i+1][0]: 
                milestone_dates.append(datetimes[i][1])
                milestone_dates.append(datetimes[i+1][0])
            i +=1  

        spans = []
        for i in range(0, len(milestone_dates)-1, 2):
            span = [milestone_dates[i], milestone_dates[i+1]]
            spans.append(span)

        print(spans)
        np.savetxt("../libra/data/apt/{0}/{1}_observable.txt".format(cycle, target), Time(spans).jd)

    planet_letters = [list('bcdef'), list('bcdef'), list('bcdefgh')]
    system_names = ['Kepler-296', 'Kepler-62', 'TRAPPIST-1']
    system_functions = [kepler296, kepler62, trappist1]

    observable_paths = glob("../libra/data/apt/{0}/*_observable.txt".format(cycle))

    eclipsing_systems = []

    cycle_start = Time('2018-10-01')

    n_observable_transits = {}

    observable_transits = {}

    for name, system, planets, path in zip(system_names, system_functions, 
                                           planet_letters, observable_paths):
        for planet in planets: 

            parameters = system(planet)

            es = EclipsingSystem(Time(parameters.t0, format='jd'), 
                                 parameters.per * u.day, 
                                 name="{0} {1}".format(name, planet))

            next_transits = es.next_primary_eclipse_time(cycle_start, n_eclipses=500)

            observable_ranges = Time(np.loadtxt(path), format='jd')

            observable_transit_times = []

            for transit in next_transits:
                for obs_range in observable_ranges: 
                    if (obs_range[0] < transit) and (obs_range[1] > transit): 
                        observable_transit_times.append(transit)

            observable_transit_times = Time(observable_transit_times)
            #print(observable_transit_times)

            observable_transits["{0} {1}".format(name, planet)] = observable_transit_times.jd.tolist()

            n_observable_transits["{0} {1}".format(name, planet)] = len(observable_transit_times)

            #eclipsing_systems.append(es)

    json.dump(observable_transits, open('../libra/data/apt/{0}/observable_transit_times.json'.format(cycle), 'w'), 
              sort_keys=True, indent=4)
    json.dump(n_observable_transits, open('../libra/data/apt/{0}/n_observable_transits.json'.format(cycle), 'w'), 
              sort_keys=True, indent=4)

[[datetime.datetime(2018, 10, 2, 0, 0), datetime.datetime(2018, 11, 12, 0, 0)], [datetime.datetime(2019, 4, 9, 0, 0), datetime.datetime(2019, 9, 30, 0, 0)]]
[[datetime.datetime(2018, 10, 2, 0, 0), datetime.datetime(2018, 10, 31, 0, 0)], [datetime.datetime(2019, 4, 2, 0, 0), datetime.datetime(2019, 9, 30, 0, 0)]]
[[datetime.datetime(2018, 10, 24, 0, 0), datetime.datetime(2018, 12, 13, 0, 0)], [datetime.datetime(2019, 6, 2, 0, 0), datetime.datetime(2019, 7, 23, 0, 0)]]


[[datetime.datetime(2019, 4, 9, 0, 0), datetime.datetime(2019, 11, 12, 0, 0)], [datetime.datetime(2020, 4, 9, 0, 0), datetime.datetime(2020, 6, 30, 0, 0)]]
[[datetime.datetime(2019, 4, 2, 0, 0), datetime.datetime(2019, 10, 31, 0, 0)], [datetime.datetime(2020, 4, 1, 0, 0), datetime.datetime(2020, 6, 30, 0, 0)]]
[[datetime.datetime(2019, 6, 2, 0, 0), datetime.datetime(2019, 7, 23, 0, 0)], [datetime.datetime(2019, 10, 25, 0, 0), datetime.datetime(2019, 12, 13, 0, 0)], [datetime.datetime(2020, 6, 1, 0, 0), datetime.datetime(2020, 6, 30, 0, 0)]]
